# Gravity
***
Create particles is an arbitrary configuration (the word "SpatialPy"), and subject them to gravity so they fall and bounce on the floor.
***
## Setup the Environment
***

In [ ]:
import os
import sys
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(), '../')))

MatPlotLib is used for creating custom visualizations

In [ ]:
import matplotlib.pyplot as plt

Numpy is used for the image data structure. Pickle is used for writing the created image file or reading the existing image file. Image, ImageDraw, and ImageFont are only used to create a domain using text.

In [ ]:
import numpy
import pickle
from PIL import Image, ImageDraw, ImageFont

In [ ]:
import gillespy3d

***
## Creating the Domain File
***
For our domain we use `True Type Font` to create an image of `SpatialPy` and convert that image into an array of particle locations for our domain.

In [ ]:
def create_ttf_image(img_pkl_file, font_path, size=(230, 70), text="SpatialPy", overwrite=False):
    # Set the font size
    fontsize = 50

    # Create the font
    fnt = ImageFont.truetype(font_path, fontsize)

    # create new image
    image = Image.new(
        # Set the mode of the image (RGB or RGBA).
        mode="RGB",

        # Set the size of the image
        size=size,

        #set the background color of the image
        color="white"
    )
    # Add the text to the image
    draw = ImageDraw.Draw(image)
    draw.text(
        # Set the top left corner of the text
        xy=(10, 10),

        # Set the text
        text=text,

        # Set the font
        font=fnt,

        # Set the fill color
        fill=(0, 0, 0)
    )

    # Write the image to a pickle file if it doesn't exist
    image_array = numpy.asarray(image)
    if not os.path.isfile(img_pkl_file) or overwrite:
        with open(img_pkl_file, 'wb') as p:
            pickle.dump(image_array, p)
    return image_array

In [ ]:
def load_ttf_image(img_pkl_file):
    # Read the existing image pickle file
    with open(img_pkl_file, 'rb') as p:
        image_array = pickle.load(p)
    return image_array

In [ ]:
def create_ttf_domain(ttf_img, preview=True):
    wrd_pnts_x = []
    wrd_pnts_y = []
    for y in range(ttf_img.shape[0]):
        for x in range(ttf_img.shape[1]):
            # Only add the letters to the domain
            if ttf_img[y, x, 0] < 50:
                wrd_pnts_x.append(x)
                # Tranform the y-coord
                wrd_pnts_y.append(-y + ttf_img.shape[0])
    if preview:
        plt.plot(wrd_pnts_x, wrd_pnts_y, '.k')
        _ = plt.axis('equal')
    return wrd_pnts_x, wrd_pnts_y

To create a domain using this method update the path to the font of your choice and change the text, size, and file name as desired, then run the cell below.

In [ ]:
args = {
    "img_pkl_file": './Domain_Files/SpatialPy_Image.pkl',
    # Font path is not the same for all computer/operating systems
    "font_path": '/usr/share/fonts/truetype/freefont/FreeSerifBold.ttf',
    "size": (230, 70),
    "text": "SpatialPy",
    "overwrite": False
}
ttf_img = create_ttf_image(**args)

To use an existing image run the cell below.

In [ ]:
ttf_img = load_ttf_image('Domain_Files/SpatialPy_Image.pkl')

Now convert the image into usable particle locations and preview the domain.

In [ ]:
wrd_pnts_x, wrd_pnts_y = create_ttf_domain(ttf_img)

***
## Creating the Boundary Conditions for the System
***
For custom boundary conditions it is necessary to subclass the `BoundaryCondition` class that implements the `__init__` and `expression` methods. The `expression` method must return a `C++` executable code block in string format.

For this example we create a custom boundry condition that simulates a particle bouncing when it interacts with the hard floor.

In [ ]:
class BouncyParticles(spatialpy.BoundaryCondition):
    def __init__(self):
        pass
    
    def expression(self):
        return """
        if(me->x[1] < system->ylo){
            me->x[1] = system->ylo;
            me->v[1] = -0.95 * me->v[1];
        }
        me->x[2] = 0.0;
        me->rho = 1.0;
        """

***
## Creating a Fluid Dynamics Model using SpatialPy
***

In [ ]:
def create_gravity_model(x_vals, y_vals, xmax, ymax, parameter_values=None):
    # Initialize Model
    model = spatialpy.Model("SpatialPy Gravity")

    # Define Domain Type IDs as constants of the Model
    model.WORD = "Word"

    """
    Create an empty domain
    - numpoints: Total number of spatial domain points.
    - xlim: Range of domain along x-axis.
    - ylim: Range of domain along y-axis.
    - zlim: Range of domain along z-axis.
    - rho0: Background density for the system.
    - c0: Speed of sound for the system.
    - P0: Background pressure for the system.
    - gravity: Acceleration of gravity for the system.
    """
    domain = spatialpy.Domain(
        numpoints=0, xlim=(0, xmax), ylim=(0, ymax), zlim=(0, 0), gravity=[0, -1, 0]
    )
    # Manually fill our domain with particles
    for i, x_val in enumerate(x_vals):
        """
        Add a single point particle to the domain space.

        - point: Spatial coordinate vertices of point to be added.
        - type_id: Particle type ID of particle to be created.
        - vol: Default volume of particle to be added.
        - mass: Default mass of particle to be added.
        - nu: Default viscosity of particle to be created.
        - c: Default artificial speed of sound of particle to be created.
        - rho: Default density of particle to be created
        - fixed: True if particle is spatially fixed, else False.
        """
        domain.add_point(
            point=[x_val, y_vals[i], 0], type_id=model.WORD, vol=1.0, mass=1.0, nu=1.0
        )

    # Set Model Domain
    model.add_domain(domain)

    # Add Boundary Conditions to Model
    model.add_boundary_condition(BouncyParticles())

    # Setting staticDomain to False allows particles to move within the system.
    model.staticDomain = False
    
    # Define Domain
    tspan = spatialpy.TimeSpan.linspace(t=50, num_points=201, timestep_size=1e-3)
    
    # Set Model Timespan
    model.timespan(tspan)
    return model

### Instantiate your Model

In [ ]:
model = create_gravity_model(wrd_pnts_x, wrd_pnts_y, ttf_img.shape[1], ttf_img.shape[0])

***
## Run the Simulation
***

In [ ]:
results = model.run()

***
## Visualizations
***
Plot the results of the simulation.

For fluid dynamics problems visualizing particle properies can be valuable, lets plot the `Type IDs` of the particles at the start of the simulation using MatPlotLib.

In [ ]:
model.domain.plot_types(use_matplotlib=True)

In [ ]:
results.plot_property(
    # Set the name of the property.
    property_name='type',
    
    # Set to True to use MatPlotLib plotting
    use_matplotlib=True,
    
    # Set the width and height of the plot (for MatPlotLib these units are in inches)
    width=15,
    height=5
)

Just like before that plot is somewhat boring, so lets plot the `Type IDs` over time using Plotly anumation.

In [ ]:
results.plot_property(
    # Set the name of the property.
    property_name='type',
    
    # Set to True to use Plotly animation
    animated=True,
    
    # Set the transition and frame durations
    t_duration=300,
    f_duration=100
)

Other properties that are available for vizualization are:
- Velocity: `v`
- Density: `rho`
- Mass: `mass`
- Viscocity: `nu`
- Boundary Volume Fraction: `bvf_phi` (non-standard SDPD: https://www.sciencedirect.com/science/article/abs/pii/S0955799721000916)

Lets plot the velocity in the `Y` direction.

In [ ]:
results.plot_property(
    # Set the name of the property.
    property_name='v',
    
    # Set the velocity direction (X: 0, Y: 1, Z: 0)
    p_ndx=1,
    
    # Set to True to use Plotly animation
    animated=True,
    
    # Set the transition and frame durations
    t_duration=300,
    f_duration=100
)

A more complete list of `results.plot_property()` arguments can be found in the [documentation](https://stochss.github.io/SpatialPy/docs/build/html/classes/spatialpy.core.html#spatialpy.core.result.Result.plot_property).